# Model skill assessment

## Simple comparison

Sometimes all your need is a simple comparison of two time series. The `fmskill.compare()` method does just that.

In [ ]:
from mikeio import Dfs0
import fmskill

### The model
Can be either a dfs0 or a DataFrame. It needs to have a single item only. 

In [ ]:
fn_mod = 'data/SW/ts_storm_4.dfs0'
df_mod = Dfs0(fn_mod).read(items=0).to_dataframe()

### The observation
Can be either a dfs0, a DataFrame or a PointObservation object. It needs to have a single item only.

In [ ]:
fn_obs = 'data/SW/eur_Hm0.dfs0'

### compare()
The compare() method will return an object that can be used for scatter plots, skill assessment, time series plots etc.

In [ ]:
c = fmskill.compare(fn_obs, df_mod)

In [ ]:
c.plot_timeseries();

## Systematic vs random errors

![](images/systematic_random_error.png)

A model is an simplified version of a natural system, such as the ocean, and as such does not reflect every detail of the natural system.

In order to validate if a model does capture the essential dynamics of the natural system, it can be helpful to classify the mismatch of the model and observations in two broad categories:
* systematic errors
* random errors

A quantitativate assesment of a model involves calculating one or more model score, skill metrics, which in varying degrees capture systematic errors, random errors or a combination.

## Metrics

**Bias** is an indication of systematic error. In the left figure above, the model has negative bias (modelled wave heights are lower thatn observed). Thus it is an indication that the model can be improved.

**Root Mean Square Error** (rmse) is a combination of systematic and random error. It is a common metric to indicate the quality of a calibrated model, but less useful to understand the potential for further calibration since it captures both systematic and random errors.

**Unbiased Root Mean Square Error** (urmse) is the unbiased version of Root Mean Square Error. Since the bias is removed, it only captures the random error.

For a complete list of possible metrics, see the [Metrics section in the FMSkill docs](https://dhi.github.io/fmskill/api.html#metrics).



To get a quantitative model skill, we use the .skill() method, which returns a table (similar to a DataFrame).

In [ ]:
c.skill()

The default is a number of common metrics, but you are free to pick your favorite metrics.

In [ ]:
c.skill(metrics=["mae","rho","lin_slope"])

A very common way to visualize model skill is to use a scatter plot.

The scatter plot includes some additional features such as a 2d histogram, a Q-Q line and a regression line, but the appearance is highly configurable.

In [ ]:
c.scatter()

In [ ]:
c.scatter(binsize=0.5, 
          show_points=False,
          xlim=[0,6], ylim=[0,6],
          title="A calibrated model!")

## Taylor diagram

A taylor diagram is a way to combine several statistics in a single plot, and is very useful to compare the skill of several models, or observations in a single plot.

In [ ]:
c.taylor()

## Elaborate comparison

In [ ]:
import numpy as np
from fmskill import PointObservation
from fmskill import ModelResult, Connector

In [ ]:
fn = 'data/SW/HKZN_local_2017_DutchCoast.dfsu'
mr = ModelResult(fn, name='HKZN_local', item=0)
mr.dfs

In [ ]:
o1 = PointObservation('data/SW/HKNA_Hm0.dfs0', item=0, x=4.2420, y=52.6887, name="HKNA")
o2 = PointObservation("data/SW/eur_Hm0.dfs0", item=0, x=3.2760, y=51.9990, name="EPL")

In [ ]:
o1.hist();

In [ ]:
o1.plot(); 

### Connecting

In [ ]:
con = Connector([o1, o2], mr)
con.observations

In [ ]:
con.plot_observation_positions();

In [ ]:
cc = con.extract()
cc

In [ ]:
cc.skill().style(precision=2)

In [ ]:
cc["EPL"].skill(metrics="mean_absolute_error")

In [ ]:
cc["HKNA"].plot_timeseries(figsize=(10,5));

In [ ]:
cc["EPL"].scatter(figsize=(8,8), show_hist=True) #, ylim=[2,8], xlim=[1,10])

In [ ]:
cc["EPL"].hist(bins=20)

In [ ]:
cc["HKNA"].scatter(binsize=0.25, cmap="viridis")

In [ ]:
con.plot_temporal_coverage();